### VGG16的直接实现方式

In [1]:
import torch
import torch.nn as nn


class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
    
        self.conv = nn.Sequential(
            # Conv 1-1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 1-2 
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 1
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 2-1
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 2-2 
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 2
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 3-1
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 3-2
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 3-3
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 3
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 4-1
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 4-2
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 4-3
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 4
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 5-1
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 5-2
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 5-3
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 5
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.dense = nn.Sequential(
            # Dense 6
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            # Dense 7
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            # Dense 8
            nn.Linear(4096, 1000))
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x

### 测试VGG16的正确性

In [2]:
model = VGG16()
print (model)

VGG16(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), paddi

In [3]:
random_image = torch.randn(1, 3, 224, 224)
out = model(random_image)
print (out.size())

torch.Size([1, 1000])


In [4]:
random_image = torch.randn(1, 3, 120, 120)
out = model(random_image)

RuntimeError: size mismatch, m1: [1 x 4608], m2: [25088 x 4096] at c:\a\w\1\s\tmp_conda_3.6_090826\conda\conda-bld\pytorch_1550394668685\work\aten\src\th\generic/THTensorMath.cpp:940

### 结构化VGG网络实现

通过配置的方式，不同的配置，生成对应的不同的网络。

In [4]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [5]:
def vgg16():
    """VGG 16-layer model (configuration "D")
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['D']))
    return model

另一个很重要的特点是：全卷积网络

In [6]:
print (vgg16())

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [7]:
def vgg19(pretrained=False, **kwargs):
    """VGG 19-layer model (configuration "E")
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['E']))
    return model

In [8]:
!pip install torchsummaryX

  Using cached https://files.pythonhosted.org/packages/48/56/9b19a7050489db0dcaba38ca013b06afa61ff3b6d41c0425592f1b4bd6ff/torchsummaryX-1.1.0-py3-none-any.whl


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='pypi.org', port=443): Read timed out. (read timeout=15)",)': /simple/torchsummaryx/


In [9]:
from torchsummaryX import summary

summary(vgg16(), torch.zeros((1, 3, 224, 224)))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Conv2d               [3, 64, 3, 3]    [1, 64, 224, 224]                 1.79                86.70
1_ReLU                             -    [1, 64, 224, 224]                    -                    -
2_Conv2d              [64, 64, 3, 3]    [1, 64, 224, 224]                36.93             1,849.69
3_ReLU                             -    [1, 64, 224, 224]                    -                    -
4_MaxPool2d                        -    [1, 64, 112, 112]                    -                    -
5_Conv2d             [64, 128, 3, 3]   [1, 128, 112, 112]                73.86               924.84
6_ReLU                             -   [1, 128, 112, 112]                    -                    -
7_Conv2d            [128, 128, 3, 3]   [1, 128, 112, 112]               147.58             1,849.69

In [10]:
summary(vgg19(), torch.zeros((1, 3, 224, 224)))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Conv2d               [3, 64, 3, 3]    [1, 64, 224, 224]                 1.79                86.70
1_ReLU                             -    [1, 64, 224, 224]                    -                    -
2_Conv2d              [64, 64, 3, 3]    [1, 64, 224, 224]                36.93             1,849.69
3_ReLU                             -    [1, 64, 224, 224]                    -                    -
4_MaxPool2d                        -    [1, 64, 112, 112]                    -                    -
5_Conv2d             [64, 128, 3, 3]   [1, 128, 112, 112]                73.86               924.84
6_ReLU                             -   [1, 128, 112, 112]                    -                    -
7_Conv2d            [128, 128, 3, 3]   [1, 128, 112, 112]               147.58             1,849.69

In [11]:
summary(vgg16(), torch.zeros((1, 3, 120, 120)))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Conv2d               [3, 64, 3, 3]    [1, 64, 120, 120]                 1.79                24.88
1_ReLU                             -    [1, 64, 120, 120]                    -                    -
2_Conv2d              [64, 64, 3, 3]    [1, 64, 120, 120]                36.93               530.84
3_ReLU                             -    [1, 64, 120, 120]                    -                    -
4_MaxPool2d                        -      [1, 64, 60, 60]                    -                    -
5_Conv2d             [64, 128, 3, 3]     [1, 128, 60, 60]                73.86               265.42
6_ReLU                             -     [1, 128, 60, 60]                    -                    -
7_Conv2d            [128, 128, 3, 3]     [1, 128, 60, 60]               147.58               530.84